In [1]:
import pyspark
from pyspark import SparkContext
# from pyspark.sql.window import Window
# from pyspark.sql import functions as F
# from pyspark.sql import types as T

import json
import pandas as pd
import numpy as np

#### Functions

In [2]:
def read_data(sqlContext, filepath):
    df = sqlContext.read.format("com.databricks.spark.csv")\
        .option("header", "true")\
        .option("inferSchema", "true") \
        .option("nullValue", "-")\
        .load(filepath)

    return df

def printdf(df,l=10):
    return df.limit(l).toPandas()

In [3]:
sc = SparkContext.getOrCreate()
sqlContext = pyspark.SQLContext(sc)

### Read Boarding Data

In [4]:
boarding_data = pd.read_json('/local/tarciso/masters/data/bus_trips/preliminary-exp-sample-data/ticketing-sample/doc1-2017051015.txt', lines=True)

In [5]:
boarding_data.head()

,CODLINHA,CODVEICULO,DATANASCIMENTO,DATAUTILIZACAO,NOMELINHA,NUMEROCARTAO,SEXO
0,000,00070,22/11/58,"09/05/17 11:05:57,000000",OPER S/LINHA,1353891,M
1,542,GA117,23/03/72,"09/05/17 17:25:14,000000",BAIRRO NOVO B,2357837,F
2,000,09053,23/03/72,"09/05/17 17:57:33,000000",OPER S/LINHA,2357837,F
3,000,09053,23/03/72,"09/05/17 17:57:28,000000",OPER S/LINHA,2357837,F
4,021,08046,26/01/72,"09/05/17 20:17:34,000000",INTERB II ANTI H,1937533,F


In [6]:
boardings_total = boarding_data.count()[0]

#### Checking number of boardings on route 000

In [7]:
boardings_000 = boarding_data[boarding_data['CODLINHA'] == '000'].count()[0]

In [8]:
print boardings_000, boardings_total, boardings_000/float(boardings_total)

134629 320292 0.420332071984


#### Looking at 000 boardings in more detail

In [9]:
boardings_000_df = boarding_data[boarding_data['CODLINHA'] == '000']

In [10]:
boardings_000_df.head(20)

,CODLINHA,CODVEICULO,DATANASCIMENTO,DATAUTILIZACAO,NOMELINHA,NUMEROCARTAO,SEXO
0,000,00070,22/11/58,"09/05/17 11:05:57,000000",OPER S/LINHA,1353891,M
2,000,09053,23/03/72,"09/05/17 17:57:33,000000",OPER S/LINHA,2357837,F
3,000,09053,23/03/72,"09/05/17 17:57:28,000000",OPER S/LINHA,2357837,F
13,000,03009,28/08/84,"09/05/17 18:57:22,000000",OPER S/LINHA,3591720,F
15,000,03031,29/12/63,"09/05/17 07:20:19,000000",OPER S/LINHA,1225547,M
17,000,05040,17/01/79,"09/05/17 15:29:11,000000",OPER S/LINHA,2699543,F
20,000,09005,14/09/76,"09/05/17 06:59:45,000000",OPER S/LINHA,2788103,F
22,000,03037,01/06/89,"09/05/17 17:00:00,000000",OPER S/LINHA,2900987,F
25,000,05056,02/07/63,"09/05/17 07:42:22,000000",OPER S/LINHA,1947358,M
26,000,09029,22/02/65,"09/05/17 07:00:15,000000",OPER S/LINHA,3543165,M


In [31]:
boardings_000_df.describe(include='all')

,CODLINHA,CODVEICULO,DATANASCIMENTO,DATAUTILIZACAO,NOMELINHA,NUMEROCARTAO,SEXO
count,134629,134629,134629,134629,134629,1.346290e+05,134629
unique,1,311,18778,45061,1,NaN,3
top,000,03014,,"09/05/17 18:12:08,000000",OPER S/LINHA,NaN,F
freq,134629,2479,1351,17,134629,NaN,81325
mean,NaN,NaN,NaN,NaN,NaN,2.998047e+06,NaN
std,NaN,NaN,NaN,NaN,NaN,1.100731e+06,NaN
min,NaN,NaN,NaN,NaN,NaN,2.286960e+05,NaN
25%,NaN,NaN,NaN,NaN,NaN,2.341897e+06,NaN
50%,NaN,NaN,NaN,NaN,NaN,3.230604e+06,NaN
75%,NaN,NaN,NaN,NaN,NaN,3.660119e+06,NaN


#### Read Line 000 Codes translation table

In [79]:
line_000_terminals = pd.read_csv('/local/tarciso/masters/data/urbs/Tubos e Terminais FINAL corresp..csv', dtype = {'COD_URBS': str})

In [80]:
line_000_terminals.head()

,LINHA,NOME,COD_URBS,GRUPO_ID,LAT,LON
0,000,TERMINAL BOQUEIRÃO,00001,14489,-25.516723,-49.230568
1,000,TERMINAL BOQUEIRÃO,00002,14489,-25.516723,-49.230568
2,000,TERMINAL BOQUEIRÃO,00003,14489,-25.516723,-49.230568
3,000,TERMINAL CARMO,00004,14485,-25.501311,-49.237825
4,000,TERMINAL CARMO,00005,14485,-25.501311,-49.237825


In [81]:
line_000_terminals.count()

LINHA       360
NOME        360
COD_URBS    360
GRUPO_ID    360
LAT         360
LON         360
dtype: int64

#### Match CODVEICULO to COD_URBS to find line 000 boarding stations

In [87]:
route_000_vehicles = boardings_000_df[['CODLINHA','CODVEICULO']].drop_duplicates()

In [88]:
route_000_vehicles.count()

CODLINHA      311
CODVEICULO    311
dtype: int64

In [89]:
route_000_vehicles.head()

,CODLINHA,CODVEICULO
0,000,00070
2,000,09053
13,000,03009
15,000,03031
17,000,05040


In [94]:
matched_000_stations = route_000_vehicles.merge(line_000_terminals, left_on='CODVEICULO', right_on='COD_URBS', how='inner')

In [95]:
matched_000_stations.count()

CODLINHA      287
CODVEICULO    287
LINHA         287
NOME          287
COD_URBS      287
GRUPO_ID      287
LAT           287
LON           287
dtype: int64

In [96]:
matched_000_stations.head(20)

,CODLINHA,CODVEICULO,LINHA,NOME,COD_URBS,GRUPO_ID,LAT,LON
0,000,00070,000,TERMINAL HAUER,00070,14487,-25.481900,-49.247078
1,000,09053,000,ESTAÇÃO ARROIO CERCADO - SENT. TERM. PINHEIRINHO,09053,41833,-25.534667,-49.282752
2,000,03009,000,TERMINAL PORTÃO,03009,14499,-25.476410,-49.292681
3,000,03031,000,ESTAÇÃO SEBASTIÃO PARANÁ - SENT. BAIRRO,03031,26104,-25.461169,-49.289857
4,000,05040,000,ESTAÇÃO PASSEIO PÚBLICO - SENT. TERM. SANTA CÂ...,05040,41773,-25.423959,-49.268338
5,000,09005,000,TUBO TERM. CAIUÁ,09005,14515,-25.483504,-49.350138
6,000,03037,000,ESTAÇÃO SILVA JARDIM - SENT. BAIRRO,03037,26105,-25.448323,-49.287777
7,000,05056,000,ESTAÇÃO ANTÔNIO LAGO - SENT. CENTRO,05056,41812,-25.387903,-49.235215
8,000,09029,000,ESTAÇÃO VITAL BRASIL - SENT. BAIRRO,09029,26121,-25.464607,-49.291302
9,000,05053,000,ESTAÇÃO HOLANDA - SENT. BAIRRO,05053,41771,-25.399555,-49.246224


#### Read Stops GTFS file

In [97]:
stops_df = pd.read_csv('/local/tarciso/data/gtfs/curitiba/stops.txt')

In [98]:
stops_df.head(40)

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,70,104505,Terminal Campina do Siqueira - 303 - Centenári...,Terminal Campina do Siqueira - Campo Comprido,-25.435724,-49.306998,NaN,NaN,0,14506.0,NaN,NaN
1,270,104905,Terminal Carmo - 030 - Interbairros III,Terminal Carmo - 030 - Interbairros III (Senti...,-25.501341,-49.237597,NaN,NaN,0,14485.0,NaN,NaN
2,276,105606,Terminal Oficinas - 030 - Interbairros III,Terminal Oficinas - 030 - Interbairros III (Se...,-25.451550,-49.214917,NaN,NaN,0,14481.0,NaN,NaN
3,299,105603,Terminal Oficinas - 030 - Interbairros III,Terminal Oficinas - 030 - Interbairros III (Se...,-25.451665,-49.215086,NaN,NaN,0,14481.0,NaN,NaN
4,308,104907,Terminal Carmo - 030 - Interbairros III,Terminal Carmo - 030 - Interbairros III (Senti...,-25.501311,-49.237825,NaN,NaN,0,14485.0,NaN,NaN
5,568,190836,"R. Dep. José Hoffmann, 80 - Vista Alegre",150 - C. Música / V.Alegre (Ponto Final),-25.408609,-49.299860,NaN,NaN,0,NaN,NaN,NaN
6,581,110312,Praça Santos Andrade - 150 - C. da Música / Vi...,Praça Santos Andrade 150 - C.Música / V. Alegre .,-25.428210,-49.265846,NaN,NaN,0,NaN,NaN,NaN
7,597,190896,"R. Eng. Agro. Lauro Klas, 106 - Pilarzinho",160 - Jd. Mercês / Guanabara (Ponto Final),-25.398410,-49.293255,NaN,NaN,0,NaN,NaN,NaN
8,616,150689,"Rua Rio de Janeiro, 1293 - Água Verde",Ponto Final 160 - Jd. Mercês / Guanabara (Sent...,-25.462635,-49.277920,NaN,NaN,0,NaN,NaN,NaN
9,662,190600,"Rua São Francisco Xavier, 132 - Pilarzinho",166 - Vila Nori (Ponto Final) 167 - Fredolin W...,-25.389211,-49.302757,NaN,NaN,0,NaN,NaN,NaN


In [99]:
terminal_stops = stops_df[stops_df['location_type'] == 1]

In [100]:
len(terminal_stops)

328

In [101]:
terminal_stops.head(40)

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
118,9475,null,Terminal Urbano Campo Largo,NaN,-25.455531,-49.524119,NaN,NaN,1,NaN,NaN,NaN
143,14415,null,Estação Tubo Santa Quitéria,NaN,-25.459128,-49.302436,NaN,NaN,1,NaN,NaN,NaN
144,14432,null,Estação Tubo Mercês,NaN,-25.422391,-49.291635,NaN,NaN,1,NaN,NaN,NaN
145,14434,null,Estação Tubo Centro Cívico - Palácio Iguaçú,NaN,-25.415392,-49.267910,NaN,NaN,1,NaN,NaN,NaN
146,14470,null,Terminal Santa Cândida,NaN,-25.377073,-49.224360,NaN,NaN,1,NaN,NaN,NaN
147,14471,null,Terminal Boa Vista,NaN,-25.393295,-49.241051,NaN,NaN,1,NaN,NaN,NaN
148,14474,null,Terminal Cabral,NaN,-25.406659,-49.252791,NaN,NaN,1,NaN,NaN,NaN
149,14476,null,Terminal Bairro Alto,NaN,-25.413047,-49.205480,NaN,NaN,1,NaN,NaN,NaN
150,14478,null,Terminal Capão da Imbuia,NaN,-25.439980,-49.221858,NaN,NaN,1,NaN,NaN,NaN
151,14481,null,Terminal Oficinas,NaN,-25.451505,-49.214964,NaN,NaN,1,NaN,NaN,NaN


#### Match terminal stop codes to line 000 terminals

In [102]:
line_000_stations_stops = matched_000_stations.merge(terminal_stops, left_on='GRUPO_ID', right_on='stop_id', how='inner')

In [103]:
len(line_000_stations_stops)

287

In [104]:
line_000_stations_stops.head(20)

,CODLINHA,CODVEICULO,LINHA,NOME,COD_URBS,GRUPO_ID,LAT,LON,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,000,00070,000,TERMINAL HAUER,00070,14487,-25.481900,-49.247078,14487,null,Terminal Hauer,NaN,-25.481927,-49.246999,NaN,NaN,1,NaN,NaN,NaN
1,000,00009,000,TERMINAL HAUER,00009,14487,-25.481900,-49.247078,14487,null,Terminal Hauer,NaN,-25.481927,-49.246999,NaN,NaN,1,NaN,NaN,NaN
2,000,00007,000,TERMINAL HAUER,00007,14487,-25.481900,-49.247078,14487,null,Terminal Hauer,NaN,-25.481927,-49.246999,NaN,NaN,1,NaN,NaN,NaN
3,000,00008,000,TERMINAL HAUER,00008,14487,-25.481900,-49.247078,14487,null,Terminal Hauer,NaN,-25.481927,-49.246999,NaN,NaN,1,NaN,NaN,NaN
4,000,09053,000,ESTAÇÃO ARROIO CERCADO - SENT. TERM. PINHEIRINHO,09053,41833,-25.534667,-49.282752,41833,null,Estação Tubo Arroio Cercado,Estação Tubo Arroio Cercado - Sitio Cercado,-25.534673,-49.282746,NaN,NaN,1,NaN,NaN,NaN
5,000,09052,000,ESTAÇÃO ARROIO CERCADO - SENT. TERM. SÍTIO CER...,09052,41833,-25.534667,-49.282752,41833,null,Estação Tubo Arroio Cercado,Estação Tubo Arroio Cercado - Sitio Cercado,-25.534673,-49.282746,NaN,NaN,1,NaN,NaN,NaN
6,000,03009,000,TERMINAL PORTÃO,03009,14499,-25.476410,-49.292681,14499,null,Terminal Portão,NaN,-25.476335,-49.292629,NaN,NaN,1,NaN,NaN,NaN
7,000,03014,000,TERMINAL PORTÃO,03014,14499,-25.476410,-49.292681,14499,null,Terminal Portão,NaN,-25.476335,-49.292629,NaN,NaN,1,NaN,NaN,NaN
8,000,03006,000,TERMINAL PORTÃO,03006,14499,-25.476410,-49.292681,14499,null,Terminal Portão,NaN,-25.476335,-49.292629,NaN,NaN,1,NaN,NaN,NaN
9,000,03007,000,TERMINAL PORTÃO,03007,14499,-25.476410,-49.292681,14499,null,Terminal Portão,NaN,-25.476335,-49.292629,NaN,NaN,1,NaN,NaN,NaN


In [105]:
line_000_codes_translation = line_000_stations_stops[['CODVEICULO','stop_id','stop_code','stop_name','stop_desc','stop_lat','stop_lon','zone_id','stop_url','location_type','parent_station','stop_timezone','wheelchair_boarding']]

In [106]:
line_000_codes_translation

,CODVEICULO,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,00070,14487,null,Terminal Hauer,NaN,-25.481927,-49.246999,NaN,NaN,1,NaN,NaN,NaN
1,00009,14487,null,Terminal Hauer,NaN,-25.481927,-49.246999,NaN,NaN,1,NaN,NaN,NaN
2,00007,14487,null,Terminal Hauer,NaN,-25.481927,-49.246999,NaN,NaN,1,NaN,NaN,NaN
3,00008,14487,null,Terminal Hauer,NaN,-25.481927,-49.246999,NaN,NaN,1,NaN,NaN,NaN
4,09053,41833,null,Estação Tubo Arroio Cercado,Estação Tubo Arroio Cercado - Sitio Cercado,-25.534673,-49.282746,NaN,NaN,1,NaN,NaN,NaN
5,09052,41833,null,Estação Tubo Arroio Cercado,Estação Tubo Arroio Cercado - Sitio Cercado,-25.534673,-49.282746,NaN,NaN,1,NaN,NaN,NaN
6,03009,14499,null,Terminal Portão,NaN,-25.476335,-49.292629,NaN,NaN,1,NaN,NaN,NaN
7,03014,14499,null,Terminal Portão,NaN,-25.476335,-49.292629,NaN,NaN,1,NaN,NaN,NaN
8,03006,14499,null,Terminal Portão,NaN,-25.476335,-49.292629,NaN,NaN,1,NaN,NaN,NaN
9,03007,14499,null,Terminal Portão,NaN,-25.476335,-49.292629,NaN,NaN,1,NaN,NaN,NaN


#### Match Line 000 stations to ticketing data

In [107]:
boardings_000_stations = boardings_000_df.merge(line_000_codes_translation, on='CODVEICULO', how='inner')

In [109]:
len(boardings_000_stations)

122864